# CDA API Exploration

Explore the [Cancer Data Aggregator (CDA)](https://datacommons.cancer.gov/cancer-data-aggregator) API to understand its data model, available tables, and query capabilities.

CDA aggregates metadata from multiple CRDC nodes:
- **GDC** (Genomic Data Commons) — genomics, sequencing
- **PDC** (Proteomic Data Commons) — proteomics
- **IDC** (Imaging Data Commons) — medical imaging
- **GC** (Genomic Commons / CDS) — additional datasets
- **ICDC** (Integrated Canine Data Commons)

In [1]:
from cdapython import (
    tables, columns, column_values, release_metadata,
    summarize_subjects, summarize_files,
    get_subject_data, get_file_data
)
import pandas as pd

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

## Available Tables

In [2]:
available_tables = tables()
print("CDA tables:", available_tables)

CDA tables: ['file', 'mutation', 'observation', 'project', 'subject', 'treatment', 'upstream_identifiers']


## Table Schemas

Explore the columns in each table.

In [3]:
all_columns = columns()
all_columns

,table,column,data_type,nullable,description
0,file,file_id,text,False,A unique identifier for this file minted by CDA. May change release-to-release. Contains no semantically reliable co...
1,file,access,text,True,Denotes controlled vs open access.
2,file,anatomic_site,text,True,One or more anatomic sites described by this file.
3,file,category,text,True,"General category of file data. In the case of DICOM series from IDC, this will be decoded from the DICOM Modality co..."
4,file,drs_uri,text,True,"A resolvable DRS URI minted for this file and maintained by DCF; prefix is ""drs://dg.4dfc:"". In the case of a DICOM ..."
...,...,...,...,...,...
59,treatment,treatment_anatomic_site,text,True,"Anatomic site targeted by this treatment, classified according to the Uber-anatomy ontology (UBERON)."
60,treatment,treatment_type,text,True,Type of treatment performed.
61,upstream_identifiers,upstream_id,text,False,"One identifier, exactly as published by the specified upstream data source."
62,upstream_identifiers,upstream_field,text,False,"What this identifier is called by the specified upstream data source, like for example ""case.submitter_id"" or ""parti..."


In [4]:
# Subject table — core demographics
columns(table='subject')

,table,column,data_type,nullable,description
0,subject,subject_id,text,False,A unique identifier for this subject minted by CDA. May change release-to-release. Contains no semantically reliable...
1,subject,cause_of_death,text,True,The clinical cause of this subject's death.
2,subject,ethnicity,text,True,This subject's self-identified ethnicity.
3,subject,race,text,True,This subject's self-identified race.
4,subject,species,text,True,Binomial (genus-species) or trinomial (genus-species-strain) scientific name of this subject's species.
5,subject,year_of_birth,integer,True,The year this subject was born. Values may be masked or transformed by upstream data managers to protect personally ...
6,subject,year_of_death,integer,True,The year this subject died. Values may be masked or transformed by upstream data managers to protect personally iden...


In [5]:
# Observation table — diagnoses, staging, morphology
columns(table='observation')

,table,column,data_type,nullable,description
0,observation,age_at_observation,integer,True,"The approximate age at which this observation was made, computed directly as (year_of_observation - subject.year_of_..."
1,observation,diagnosis,text,True,"A disease with which this subject was diagnosed, classified according to the World Health Organization's ICD-O-3 cod..."
2,observation,grade,text,True,A clinical characterization for this subject of how well-differentiated a tumor is from surrounding tissue.
3,observation,morphology,text,True,"A clinical characterization for this subject of tumor morphology, classified according to the World Health Organizat..."
4,observation,observed_anatomic_site,text,True,"Anatomic site observed, classified according to the Uber-anatomy ontology (UBERON)."
5,observation,resection_anatomic_site,text,True,"Anatomic site from which material was sampled or removed, classified according to the Uber-anatomy ontology (UBERON)."
6,observation,sex,text,True,Sex of the subject.
7,observation,stage,text,True,"An estimate for this subject of the extent to which a cancer has grown and spread, classified according to the Ameri..."
8,observation,vital_status,text,True,"Vital status (at present, values are restricted to ""alive"" or ""dead"") of the subject."
9,observation,year_of_observation,integer,True,The year this observation was made. Values may be masked or transformed by upstream data managers to protect persona...


In [6]:
# File table — data files across all nodes
columns(table='file')

,table,column,data_type,nullable,description
0,file,file_id,text,False,A unique identifier for this file minted by CDA. May change release-to-release. Contains no semantically reliable co...
1,file,access,text,True,Denotes controlled vs open access.
2,file,anatomic_site,text,True,One or more anatomic sites described by this file.
3,file,category,text,True,"General category of file data. In the case of DICOM series from IDC, this will be decoded from the DICOM Modality co..."
4,file,drs_uri,text,True,"A resolvable DRS URI minted for this file and maintained by DCF; prefix is ""drs://dg.4dfc:"". In the case of a DICOM ..."
5,file,file_description,text,True,Optional text describing this file.
6,file,file_name,text,True,File name or label (may optionally include local path prefix; this will not generally be a URI). In the case of DICO...
7,file,file_type,text,True,"File data type (like ""CT image"" or ""miRNA expression quantification""). In the case of a DICOM series from IDC, this ..."
8,file,format,text,True,"File format (like JPEG, PDF or CSV)."
9,file,size,bigint,True,"File size in bytes. In the case of a DICOM series from IDC, the value in this field will be the sum of the sizes of ..."


In [7]:
# Treatment table
columns(table='treatment')

,table,column,data_type,nullable,description
0,treatment,therapeutic_agent,text,True,A substance (medication) administered as part of this treatment.
1,treatment,treatment_anatomic_site,text,True,"Anatomic site targeted by this treatment, classified according to the Uber-anatomy ontology (UBERON)."
2,treatment,treatment_type,text,True,Type of treatment performed.


In [8]:
# Upstream identifiers — the key to cross-node linking
columns(table='upstream_identifiers')

,table,column,data_type,nullable,description
0,upstream_identifiers,upstream_id,text,False,"One identifier, exactly as published by the specified upstream data source."
1,upstream_identifiers,upstream_field,text,False,"What this identifier is called by the specified upstream data source, like for example ""case.submitter_id"" or ""parti..."
2,upstream_identifiers,upstream_source,text,False,The upstream data source from which this identifier was extracted by CDA.


## Data Sources Overview

What upstream data sources does CDA index?

In [9]:
column_values('upstream_source')

,upstream_source,value_count
0,dbGaP,188
1,ICDC,2812
2,PDC,11248
3,GDC,100779
4,GC,144852
5,IDC,239289


In [10]:
# What upstream identifier fields exist?
column_values('upstream_field')

,upstream_field,value_count
0,program.uuid,6
1,Program.name,7
2,Program.program_id,7
3,Program.program_submitter_id,7
4,program.dbgap_accession_number,11
5,program.program_acronym,11
6,program.program_name,11
7,case.canine_individual_id,18
8,study.clinical_study_designation,18
9,study.clinical_study_name,18


## Release Metadata

Statistics about the current CDA release.

In [11]:
metadata = release_metadata()
metadata_df = pd.DataFrame(metadata)
metadata_df

,cda_table,cda_column,data_source,data_source_version,data_source_extraction_date,data_source_row_count,data_source_unique_value_count,data_source_null_count
0,file,access,CDA,December 2025,2025-12-08,3329989,2,490223
1,file,access,GC,21.0,2025-10-20,392644,2,0
2,file,access,GDC,"Data Release 45.0 - December 04, 2025",2025-12-05,1761673,2,490223
3,file,access,ICDC,2023-10-16,2025-12-08,2895,1,0
4,file,access,IDC,v23,2025-11-26,994073,1,0
...,...,...,...,...,...,...,...,...
146,treatment,therapeutic_agent,CDA,December 2025,2025-12-08,58832,369,37624
147,treatment,therapeutic_agent,GC,21.0,2025-10-20,453,4,0
148,treatment,therapeutic_agent,GDC,"Data Release 45.0 - December 04, 2025",2025-12-05,58332,369,37577
149,treatment,therapeutic_agent,ICDC,2023-10-16,2025-12-08,28,0,28


In [12]:
# Summary by data source
source_summary = metadata_df.groupby('data_source').agg(
    tables=('cda_table', 'nunique'),
    total_rows=('data_source_row_count', 'sum')
).reset_index()
source_summary

,data_source,tables,total_rows
0,CDA,6,62672556
1,GC,5,3332518
2,GDC,6,50634608
3,ICDC,5,40137
4,IDC,4,7593806
5,PDC,5,1165645


## Overall Subject Summary

In [13]:
summarize_subjects()

╔═══════════════════════════════╗
║ number_of_matching_subjects   ║
╠═══════════════════════════════╣
║ 173111                        ║
╚═══════════════════════════════╝
╔════════════════════════════════════════════════╗
║ number_of_files_related_to_matching_subjects   ║
╠════════════════════════════════════════════════╣
║ 3326681                                        ║
╚════════════════════════════════════════════════╝
╔════════════╦══════════════════════╗
║   subjects ║          data_source ║
╠════════════╬══════════════════════╣
║      63618 ║             IDC only ║
║      55238 ║              GC only ║
║      34164 ║             GDC only ║
║       9706 ║            IDC + GDC ║
║       4122 ║       IDC + GDC + GC ║
║       2758 ║             PDC only ║
║       1106 ║ IDC + GDC + GC + PDC ║
║        949 ║            ICDC only ║
║        765 ║      IDC + GDC + PDC ║
║        265 ║            GDC + PDC ║
║        211 ║             IDC + GC ║
║         80 ║           IDC + ICDC ║
║    

In [14]:
# Subjects by data source
for src in ['GDC', 'PDC', 'IDC']:
    print(f"\n--- {src} ---")
    summarize_subjects(data_source=src)


--- GDC ---


╔═══════════════════════════════╗
║ number_of_matching_subjects   ║
╠═══════════════════════════════╣
║ 50213                         ║
╚═══════════════════════════════╝
╔════════════════════════════════════════════════╗
║ number_of_files_related_to_matching_subjects   ║
╠════════════════════════════════════════════════╣
║ 2027621                                        ║
╚════════════════════════════════════════════════╝
╔════════════╦══════════════════════╗
║   subjects ║          data_source ║
╠════════════╬══════════════════════╣
║      34164 ║             GDC only ║
║       9706 ║            IDC + GDC ║
║       4122 ║       IDC + GDC + GC ║
║       1106 ║ IDC + GDC + GC + PDC ║
║        765 ║      IDC + GDC + PDC ║
║        265 ║            GDC + PDC ║
║         44 ║       GDC + GC + PDC ║
║         41 ║             GDC + GC ║
╚════════════╩══════════════════════╝
╔════════════════╦════════════════════╗
║   count_result ║          ethnicity ║
╠════════════════╬════════════════════╣

╔═══════════════════════════════╗
║ number_of_matching_subjects   ║
╠═══════════════════════════════╣
║ 4982                          ║
╚═══════════════════════════════╝
╔════════════════════════════════════════════════╗
║ number_of_files_related_to_matching_subjects   ║
╠════════════════════════════════════════════════╣
║ 363080                                         ║
╚════════════════════════════════════════════════╝
╔════════════╦══════════════════════╗
║   subjects ║          data_source ║
╠════════════╬══════════════════════╣
║       2758 ║             PDC only ║
║       1106 ║ IDC + GDC + GC + PDC ║
║        765 ║      IDC + GDC + PDC ║
║        265 ║            GDC + PDC ║
║         44 ║       GDC + GC + PDC ║
║         30 ║            IDC + PDC ║
║         11 ║       IDC + GC + PDC ║
║          3 ║             GC + PDC ║
╚════════════╩══════════════════════╝
╔════════════════╦════════════════════╗
║   count_result ║          ethnicity ║
╠════════════════╬════════════════════╣

╔═══════════════════════════════╗
║ number_of_matching_subjects   ║
╠═══════════════════════════════╣
║ 79649                         ║
╚═══════════════════════════════╝
╔════════════════════════════════════════════════╗
║ number_of_files_related_to_matching_subjects   ║
╠════════════════════════════════════════════════╣
║ 2239722                                        ║
╚════════════════════════════════════════════════╝
╔════════════╦══════════════════════╗
║   subjects ║          data_source ║
╠════════════╬══════════════════════╣
║      63618 ║             IDC only ║
║       9706 ║            IDC + GDC ║
║       4122 ║       IDC + GDC + GC ║
║       1106 ║ IDC + GDC + GC + PDC ║
║        765 ║      IDC + GDC + PDC ║
║        211 ║             IDC + GC ║
║         80 ║           IDC + ICDC ║
║         30 ║            IDC + PDC ║
║         11 ║       IDC + GC + PDC ║
╚════════════╩══════════════════════╝
╔════════════════╦════════════════════╗
║   count_result ║          ethnicity ║
╠

## Test: Cross-Node Query for a Known TCGA Case

Use `upstream_id` to find a TCGA subject across all CRDC nodes. The `data_source` column in the result indicates which nodes have data.

In [15]:
# Query by upstream_id (works for TCGA barcodes = IDC PatientID)
test_subject = get_subject_data(match_all=['upstream_id = TCGA-DD-A4NO'])

for col in test_subject.columns:
    print(f"{col}: {test_subject[col].iloc[0]}")

subject_id: TCGA.TCGA-DD-A4NO
cause_of_death: <NA>
ethnicity: Non-Hispanic
race: White
species: human
year_of_birth: <NA>
year_of_death: <NA>
data_source: ['GDC', 'IDC']
upstream_id: ['75f78c38-6854-4390-ad11-789b4e549d63', 'TCGA-DD-A4NO', 'e33806d9-19ad-48b4-a190-7af7b1d7da27']


The `data_source` field shows which CRDC nodes have data for this subject.

## Test: File Data for Cross-Node Subject

Use `subject_id` (from the subject query) to find all files across nodes.

In [16]:
# Get files — use subject_id for file queries
subject_id = test_subject['subject_id'].iloc[0]
print(f"Querying files for subject_id: {subject_id}")

files = get_file_data(match_all=[f'subject_id = {subject_id}'])
print(f"Total files: {len(files)}")
print(f"Columns: {list(files.columns)}")

Querying files for subject_id: TCGA.TCGA-DD-A4NO


Total files: 184
Columns: ['file_id', 'access', 'anatomic_site', 'category', 'drs_uri', 'file_description', 'file_name', 'file_type', 'format', 'size', 'tumor_vs_normal', 'data_source', 'subject_id']


In [17]:
# Breakdown by data source and category
exploded = files.explode('data_source')
breakdown = exploded.groupby(['data_source', 'category']).agg(
    files=('file_id', 'count'),
    total_size_MB=('size', lambda x: round(x.sum() / 1e6, 1))
).reset_index()
breakdown

,data_source,category,files,total_size_MB
0,GDC,<NA>,10,0.6
1,GDC,Biospecimen,15,2113.0
2,GDC,Copy Number Variation,16,500.1
3,GDC,DNA Methylation,3,29.3
4,GDC,Proteome Profiling,1,0.0
5,GDC,Sequencing Reads,17,559686.2
6,GDC,Simple Nucleotide Variation,31,71.4
7,GDC,Somatic Structural Variation,5,0.3
8,GDC,Structural Variation,5,0.1
9,GDC,Transcriptome Profiling,4,6.7


## Key Findings

**Query patterns for cross-node analysis:**

1. **Find subject across nodes:** `get_subject_data(match_all=['upstream_id = <PatientID>'])` → check `data_source` column
2. **Get subject's CDA ID:** Extract `subject_id` from result (format: `TCGA.TCGA-XX-XXXX`)
3. **Find all files across nodes:** `get_file_data(match_all=['subject_id = <subject_id>'])` → group by `data_source` and `category`
4. **Batch query:** Use `match_from_file` parameter with a CSV of patient IDs

**CDA tables relevant for cross-node enrichment:**
- `subject` — demographics (race, ethnicity, species)
- `observation` — diagnosis, grade, morphology, stage, vital status
- `file` — files across all CRDC nodes with category, type, format, size
- `treatment` — therapeutic agents, treatment types
- `mutation` — somatic mutations (from GDC)
- `upstream_identifiers` — cross-reference IDs between nodes